In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

def extended_lines(lines, line_image, imshape):
    """
    This function draw a single, solid line over the left lane line 
    and a single solid line over the right lane line 
    """
    ymin = imshape[0]
    ymax = imshape[0]
    left_lane = []
    right_lane = []
    left_y = []
    right_y = []
    left_x = []
    right_x = []

    for line in lines:
        for x1,y1,x2,y2 in line:
            lane, intercept = np.polyfit((x1,x2), (y1,y2), 1)
            ymin = min(min(y1,y2), ymin)

            if(lane > 0):
                left_lane += [lane]
                left_y += [y1,y2]
                left_x += [x1,x2]
            else:
                right_lane += [lane]
                right_y += [y1,y2]
                right_x += [x1,x2]

        left_avg = np.mean(left_lane)
        left_y_avg = np.mean(left_y)
        left_x_avg = np.mean(left_x)
        left = left_y_avg - (left_avg * left_x_avg)

        right_avg = np.mean(right_lane)
        right_y_avg = np.mean(right_y)
        right_x_avg = np.mean(right_x)
        right = right_y_avg - (right_avg * right_x_avg)

        if ((len(left_lane) > 0) and (len(right_lane) > 0)):
            upper_left_x = int((ymin - left) / left_avg)
            lower_left_x = int((ymax - left) / left_avg)
            upper_right_x = int((ymin - right) / right_avg)
            lower_right_x = int((ymax - right) / right_avg)

            cv2.line(line_image, (upper_left_x, ymin), (lower_left_x, ymax), color=[255,0,0], thickness = 6)
            cv2.line(line_image, (upper_right_x, ymin), (lower_right_x, ymax), color=[255,0,0], thickness = 6)


    return line_image

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    return extended_lines(lines, line_img, line_img.shape)

In [ ]:
class yellow_params:
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 4 # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 20 #minimum number of pixels making up a line
    max_line_gap = 1    # maximum gap in pixels between connectable line segments

class white_params:
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15 # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 20 #minimum number of pixels making up a line
    max_line_gap = 40    # maximum gap in pixels between connectable line segments

In [ ]:
def process_image(frame):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    imshape = frame.shape
    # Here is the pipeline
    grayscale_img = grayscale(frame)
    after_gaussian_blur = gaussian_blur(grayscale_img, 3)
    after_canny_img = canny(after_gaussian_blur,50, 150)
    vertices = np.array([[(180,imshape[0]),(440, 330), (540, 330), (imshape[1],imshape[0])]], dtype=np.int32)
    after_region_of_interest = region_of_interest(after_canny_img, vertices)
    after_hough_lines = hough_lines(after_region_of_interest, params.rho, params.theta, params.threshold, params.min_line_length, params.max_line_gap)
    #plt.imshow(after_hough_lines, cmap='gray')
    after_weighted_img = weighted_img(after_hough_lines, frame)

    return after_weighted_img

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip1 = VideoFileClip("test_videos/solidYellowLeft.mp4")
params = yellow_params()
yellow_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
params = white_params()
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)